In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
dbutils.widgets.text("target_dataset", "VCM_DWH_PRD")
dbutils.widgets.text("target_table", "f_delivery_order")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text(
    "dependency_table","VCM_STG_INF.SAP_VCCSDDL51"
)
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_delivery_order
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)  
AS
WITH
do_header AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl51
    QUALIFY ROW_NUMBER() OVER(PARTITION BY do_number ORDER BY proc_date DESC) = 1
),
do_detail AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_staging_delivery_order
    WHERE IFNULL(deleted_record, '') <> 'Y'
)
SELECT
    a.creation_date,
    a.calday,
    a.do_number,
    a.do_item,
    a.gi_date,
    a.act_gi_dte,
    a.product_id,
    a.deliver_qty_sale,
    a.sales_unit,
    a.deliver_qty_base,
    a.base_unit,
    a.dc_id,
    b.store_id,
    a.sto_number,
    a.sto_item,
    a.sts_pick,
    a.osts_pick,
    a.tgoodmv_st,
    a.pick_conf,
    a.goodsmv_st,
    a.recordmode,
    a.unload_pt,
    a.imounl_pt,
    a.pick_indc,
    a.del_type,
    a.ship_date,
    a.createdby,
    a.crea_time
FROM do_detail a
LEFT JOIN do_header b ON a.do_number = b.do_number;
""")